In [1]:
using CSV, DataFrames,Plots,Random,PlotlyJS, ClassImbalance, GLM, Lathe, Statistics, ScikitLearn

WebIO._IJuliaInit()

In [5]:
@sk_import linear_model: LogisticRegression;
@sk_import model_selection:train_test_split;
@sk_import preprocessing:StandardScaler;
@sk_import decomposition:PCA;
using ScikitLearn.CrossValidation: cross_val_score;

┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore /home/alchemistdude/.julia/packages/ScikitLearn/Kn82b/src/Skcore.jl:169


In [3]:
df = CSV.read("data/sonar.csv",DataFrame,header=0); #Creating dataframe object from csv file
df.Column61 .= replace.(df.Column61, "M" => 1)
df.Column61 .= replace.(df.Column61, "R" => 0);
df.Column61 = parse.(Float64, df.Column61);

Podemos observar que el dataframe tiene demasiadas caracteristicas, 61 columnas para ser más exacto, utilizaremos PCA y ajustaremos un modelo con n componentes utilizando PCA, donde 1<=n<=60, posteriormente ajustaremos los modelos.

In [24]:
function best_PCA(df,model)
    
    value=0
    n_c=0
    best_acc = 0
    best_vacc = 0
    best_crossval_acc=0
    y_f=[]
    x_f = []
    
    
    
    for i in 1:60 #Miraremos PCA para valores de 1 a 60
       
        scaler = StandardScaler(); # Creamos objetos estandarizador
        X = df[:,1:60]          
        X = convert(Matrix,X)
        scaler.fit(X)
        X = scaler.transform(X); # Estandarizamos los datos
        y = df.Column61;

        pca = PCA(convert(Int64,i))
        pca.fit(X)
        X = pca.transform(X);  # Reducimos la dimensionalidad en i atributos
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42);#particion test/train
        model.fit(X_train, y_train); # Ajustamos el modelo
        
        accuracy = score(model, X_test, y_test) # Miramos el accuracy del modelo
        
        m = mean(cross_val_score(model, X, y; cv=15)) # Promedio de los accuracy de la validacion cruzada

        print("PCA = ",pca.n_components_,"-> accuracy = ",accuracy,"\n")

        if accuracy > value
            value = accuracy
            # datos del mejor modelo
            best_acc = accuracy # accuracy del mejor modelo
            x_f = X # datos despues del PCA con dimensionalidad(208,i)
            y_f = y
            n_c =pca.n_components_ # numeros de componentes

        end
    end
    
    return best_acc,x_f,y_f,n_c
end


best_PCA (generic function with 1 method)

# Linear regression

In [25]:
@sk_import linear_model: LinearRegression;
model_lr = LinearRegression();
best_acc_lr,x_f_lr,y_f_lr ,n_c_lr= best_PCA(df,model_lr);

PCA = 1-> accuracy = 0.059562414540473174
PCA = 2-> accuracy = 0.15624077819699744
PCA = 3-> accuracy = 0.3307364521092838
PCA = 4-> accuracy = 0.33095810984898966
PCA = 5-> accuracy = 0.32606886197874474
PCA = 6-> accuracy = 0.3298596671019368
PCA = 7-> accuracy = 0.3654538909248488
PCA = 8-> accuracy = 0.3600242326133668
PCA = 9-> accuracy = 0.3545340947430565
PCA = 10-> accuracy = 0.29319940102401054
PCA = 11-> accuracy = 0.29851968331000056
PCA = 12-> accuracy = 0.2982454754275813
PCA = 13-> accuracy = 0.3001490741510593
PCA = 14-> accuracy = 0.3261672146829627
PCA = 15-> accuracy = 0.3283083604406082
PCA = 16-> accuracy = 0.3359689521157221
PCA = 17-> accuracy = 0.3042533714699044
PCA = 18-> accuracy = 0.3095348047578682
PCA = 19-> accuracy = 0.31006885092388037
PCA = 20-> accuracy = 0.30656558629284736
PCA = 21-> accuracy = 0.3044848862260048
PCA = 22-> accuracy = 0.25084285495000047
PCA = 23-> accuracy = 0.21508916936682376
PCA = 24-> accuracy = 0.22143968577571138
PCA = 25-> ac

In [26]:
n_c_lr

7

In [27]:
best_acc_lr

0.3654538909248488

Para una regresión linea con hyperparametros por defecto, obtenemos que el número de componentes que ofrecen el mejor accuracy después de un PCA son 7 de 60 originalmente.Construiremos el modelo:

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x_f_lr, y_f_lr, test_size=0.35, random_state=42);#particion test/train

In [ ]:
model_lr = LinearRegression();
model_lr.fit(X_train, y_train); # Ajustamos el modelo

Obtenemos un accuracy del:

In [34]:
accuracy = score(model_lr, X_test, y_test)

0.3654538909248488

Ahora realizamos validación cruzada con k=10:

In [36]:
cross_val_score(model_lr, x_f_lr, y_f_lr; cv=10)

10-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
 -0.617592508412488
  0.0
  0.0
  0.0
  0.0
  0.0

Concluimos que una regresión lineal para un problema de clasificación da unos resultados pesimos.

# Logistic regression

In [40]:
model_log = LogisticRegression();
best_acc_log,x_f_log,y_f_log ,n_c_log= best_PCA(df,model_log);

PCA = 1-> accuracy = 0.589041095890411
PCA = 2-> accuracy = 0.6301369863013698
PCA = 3-> accuracy = 0.7808219178082192
PCA = 4-> accuracy = 0.7671232876712328
PCA = 5-> accuracy = 0.7534246575342466
PCA = 6-> accuracy = 0.7808219178082192
PCA = 7-> accuracy = 0.7945205479452054
PCA = 8-> accuracy = 0.7671232876712328
PCA = 9-> accuracy = 0.7671232876712328
PCA = 10-> accuracy = 0.7808219178082192
PCA = 11-> accuracy = 0.7534246575342466
PCA = 12-> accuracy = 0.7534246575342466
PCA = 13-> accuracy = 0.726027397260274
PCA = 14-> accuracy = 0.7534246575342466
PCA = 15-> accuracy = 0.7534246575342466
PCA = 16-> accuracy = 0.7534246575342466
PCA = 17-> accuracy = 0.7671232876712328
PCA = 18-> accuracy = 0.7808219178082192
PCA = 19-> accuracy = 0.7534246575342466
PCA = 20-> accuracy = 0.7945205479452054
PCA = 21-> accuracy = 0.7808219178082192
PCA = 22-> accuracy = 0.7671232876712328
PCA = 23-> accuracy = 0.7534246575342466
PCA = 24-> accuracy = 0.7808219178082192
PCA = 25-> accuracy = 0.780

In [41]:
n_c_log # Número de dimensiones

7

In [42]:
best_acc_log # Accuracy del modelo

0.7945205479452054

Para una regresión logística con hyperparametros por defecto, obtenemos que el número de componentes que ofrecen el mejor accuracy después de un PCA son 7 de 60 originalmente.Construiremos el modelo:

In [45]:
X_train, X_test, y_train, y_test = train_test_split(x_f_log, y_f_log, test_size=0.35, random_state=42);

In [47]:
model_log = LogisticRegression();
model_log.fit(X_train,y_train);

Obtenemos un accuracy del:

In [48]:
accuracy = score(model_log, X_test, y_test)

0.7945205479452054

Ahora realizamos validación cruzada con k=10:

In [49]:
cross_val_score(model_log, x_f_log, y_f_log; cv=10)

10-element Vector{Float64}:
 0.42857142857142855
 0.7619047619047619
 0.7142857142857143
 0.6666666666666666
 0.47619047619047616
 0.47619047619047616
 1.0
 0.3333333333333333
 0.65
 0.5

Concluimos que nuestro modelo de regresión logística tiene un poco de overfitting.

## Support Vector Machines

In [51]:
@sk_import svm: SVC;

In [52]:
model_svc = SVC(gamma="auto");
best_acc_svc,x_f_svc,y_f_svc,n_svc= best_PCA(df,model_svc);

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

...

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

PCA = 1-> accuracy = 0.5616438356164384
PCA = 2-> accuracy = 0.6301369863013698
PCA = 3-> accuracy = 0.7397260273972602
PCA = 4-> accuracy = 0.821917808219178
PCA = 5-> accuracy = 0.863013698630137
PCA = 6-> accuracy = 0.863013698630137
PCA = 7-> accuracy = 0.8767123287671232
PCA = 8-> accuracy = 0.8904109589041096
PCA = 9-> accuracy = 0.9041095890410958
PCA = 10-> accuracy = 0.9041095890410958
PCA = 11-> accuracy = 0.9315068493150684
PCA = 12-> accuracy = 0.9315068493150684
PCA = 13-> accuracy = 0.9178082191780822
PCA = 14-> accuracy = 0.9178082191780822
PCA = 15-> accuracy = 0.9178082191780822
PCA = 16-> accuracy = 0.9178082191780822
PCA = 17-> accuracy = 0.9178082191780822
PCA = 18-> accuracy = 0.9041095890410958
PCA = 19-> accuracy = 0.9041095890410958
PCA = 20-> accuracy = 0.9041095890410958
PCA = 21-> accuracy = 0.9041095890410958
PCA = 22-> accuracy = 0.9041095890410958
PCA = 23-> accuracy = 0.9041095890410958
PCA = 24-> accuracy = 0.9041095890410958
PCA = 25-> accuracy = 0.9178

In [53]:
n_svc

11

In [54]:
best_acc_svc

0.9315068493150684

Para un modelo de SVM con hyperparametros por defecto, obtenemos que el número de componentes que ofrecen el mejor accuracy después de un PCA son 11 de 60 originalmente.Construiremos el modelo:

In [55]:
X_train, X_test, y_train, y_test = train_test_split(x_f_svc, y_f_svc, test_size=0.35, random_state=42);

In [58]:
model_svc.fit(X_train,y_train);

Obtenemos un accuracy del:

In [59]:
accuracy = score(model_svc, X_test, y_test)

0.9315068493150684

Ahora realizamos validación cruzada con k=10:

In [60]:
cross_val_score(model_svc, x_f_svc, y_f_svc; cv=10)

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

10-element Vector{Float64}:
 0.3333333333333333
 0.5238095238095238
 0.6190476190476191
 0.5714285714285714
 0.5238095238095238
 0.2857142857142857
 0.19047619047619047
 0.09523809523809523
 0.6
 0.55

Concluimos que nuestro modelo de SVC tiene bastante overfitting.

# Decision Tree Classifier

In [62]:
@sk_import tree:DecisionTreeClassifier;

In [63]:
model_dtc = DecisionTreeClassifier();
best_acc_dtc,x_f_dtc,y_f_dtc,n_dtc= best_PCA(df,model_dtc);

PCA = 1-> accuracy = 0.5068493150684932
PCA = 2-> accuracy = 0.6301369863013698
PCA = 3-> accuracy = 0.7671232876712328
PCA = 4-> accuracy = 0.7945205479452054
PCA = 5-> accuracy = 0.8082191780821918
PCA = 6-> accuracy = 0.7397260273972602
PCA = 7-> accuracy = 0.821917808219178
PCA = 8-> accuracy = 0.8356164383561644
PCA = 9-> accuracy = 0.7808219178082192
PCA = 10-> accuracy = 0.7671232876712328
PCA = 11-> accuracy = 0.7945205479452054
PCA = 12-> accuracy = 0.8082191780821918
PCA = 13-> accuracy = 0.7945205479452054
PCA = 14-> accuracy = 0.7808219178082192
PCA = 15-> accuracy = 0.7808219178082192
PCA = 16-> accuracy = 0.7808219178082192
PCA = 17-> accuracy = 0.7808219178082192
PCA = 18-> accuracy = 0.7671232876712328
PCA = 19-> accuracy = 0.7945205479452054
PCA = 20-> accuracy = 0.7945205479452054
PCA = 21-> accuracy = 0.7397260273972602
PCA = 22-> accuracy = 0.7945205479452054
PCA = 23-> accuracy = 0.7671232876712328
PCA = 24-> accuracy = 0.8082191780821918
PCA = 25-> accuracy = 0.73

In [64]:
n_dtc

8

In [65]:
best_acc_dtc

0.8356164383561644

Para un modelo de arboles de decisión con hyperparametros por defecto, obtenemos que el número de componentes que ofrecen el mejor accuracy después de un PCA son 7 de 60 originalmente.Construiremos el modelo:

In [66]:
X_train, X_test, y_train, y_test = train_test_split(x_f_dtc, y_f_dtc, test_size=0.35, random_state=42);

In [68]:
model_dtc.fit(X_train,y_train);

Obtenemos un accuracy del:

In [69]:
accuracy = score(model_dtc, X_test, y_test)

0.8493150684931506

Ahora realizamos validación cruzada con k=10:

In [70]:
cross_val_score(model_dtc, x_f_dtc, y_f_dtc; cv=10)

10-element Vector{Float64}:
 0.47619047619047616
 0.5238095238095238
 0.7619047619047619
 0.7619047619047619
 0.2857142857142857
 0.5714285714285714
 0.6666666666666666
 0.42857142857142855
 0.6
 1.0

Concluimos que este modelo tiene muy poco overfitting.

# KNN

In [71]:
@sk_import neighbors:KNeighborsClassifier;

In [73]:
model_knn = KNeighborsClassifier()
best_acc_knn,x_f_knn,y_f_knn,n_knn= best_PCA(df,model_knn);

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

...

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

PCA = 1-> accuracy = 0.6027397260273972
PCA = 2-> accuracy = 0.6164383561643836
PCA = 3-> accuracy = 0.7534246575342466
PCA = 4-> accuracy = 0.8356164383561644
PCA = 5-> accuracy = 0.8493150684931506
PCA = 6-> accuracy = 0.863013698630137
PCA = 7-> accuracy = 0.8767123287671232
PCA = 8-> accuracy = 0.9041095890410958
PCA = 9-> accuracy = 0.8767123287671232
PCA = 10-> accuracy = 0.8767123287671232
PCA = 11-> accuracy = 0.863013698630137
PCA = 12-> accuracy = 0.8904109589041096
PCA = 13-> accuracy = 0.863013698630137
PCA = 14-> accuracy = 0.8767123287671232
PCA = 15-> accuracy = 0.863013698630137
PCA = 16-> accuracy = 0.8356164383561644
PCA = 17-> accuracy = 0.8493150684931506
PCA = 18-> accuracy = 0.821917808219178
PCA = 19-> accuracy = 0.8356164383561644
PCA = 20-> accuracy = 0.8356164383561644
PCA = 21-> accuracy = 0.8493150684931506
PCA = 22-> accuracy = 0.863013698630137
PCA = 23-> accuracy = 0.863013698630137
PCA = 24-> accuracy = 0.8493150684931506
PCA = 25-> accuracy = 0.86301369

In [74]:
n_knn

8

In [75]:
best_acc_knn

0.9041095890410958

In [669]:
best_crossval_acc_knn

0.4761904761904761

Para un modelo de arboles de KNN con hyperparametros por defecto, obtenemos que el número de componentes que ofrecen el mejor accuracy después de un PCA son 8 de 60 originalmente.Construiremos el modelo:

In [76]:
X_train, X_test, y_train, y_test = train_test_split(x_f_knn, y_f_knn, test_size=0.35, random_state=42);

In [77]:
model_knn.fit(X_train,y_train);

Obtenemos un accuracy del:

In [78]:
accuracy = score(model_knn, X_test, y_test)

0.9041095890410958

In [79]:
cross_val_score(model_knn, x_f_knn, y_f_knn; cv=10)

/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/alchemistdude/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `_pairwise` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(m

10-element Vector{Float64}:
 0.7142857142857143
 0.7142857142857143
 0.5238095238095238
 0.42857142857142855
 0.5238095238095238
 0.3333333333333333
 0.38095238095238093
 0.3333333333333333
 0.55
 0.35

Concluimos que este modelo tiene un poco de overfitting

# Mejor modelo : Decision Tree Classifier

Podemos concluir que el mejor modelo es el de arbol de decisión, tiene un sobreajuste muy mínimo, ademas esta por encima encima de la linea base de accuracy del 0.53